<a href="https://colab.research.google.com/github/yashx07/HuggingFaceRagAgent/blob/main/HuggingRag2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken
!pip install langchain openai

!pip install langchain-community
!pip install unstructured
!pip install pdfminer.six
!pip install langchain
!pip install openai



In [ ]:
pip install langchain-community
pip install chromadb


In [ ]:
import tiktoken
print(tiktoken.__version__)


In [ ]:
pip install unstructured[local-inference]


In [47]:
from langchain.embeddings import HuggingFaceEmbeddings


In [48]:
from langchain_community.document_loaders import OnlinePDFLoader


In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
from google.colab import userdata
userdata.get('HUGGINGFACEHUB_API_TOKEN')
userdata.get('LANGCHAIN_API_KEY')


In [65]:
import os
import numpy as np
import tiktoken
from langchain_community.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set environment variables
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ""


In [66]:
# Load the document
# pdf_url = "https://www.cet.edu.in/noticefiles/271_AI%20Lect%20Notes.pdf"
loader = OnlinePDFLoader(pdf_url)
documents = loader.load()

# Text splitting
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=128, encoding_name="cl100k_base"
)
splits = text_splitter.split_documents(documents)


In [67]:
def count_tokens(text: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))
def truncate_context(context: str, max_length: int = 1024) -> str:
    """Truncates context to fit within a maximum length."""
    return context[:max_length]



In [68]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    api_key="HUGGINGFACEHUB_API_TOKEN"
)


In [69]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [ ]:
pip install -U langchain-huggingface


In [ ]:

# Correct LLM Setup
from langchain_huggingface import HuggingFacePipeline  # Updated import
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline


model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

text2text_pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=text2text_pipe)


In [72]:
prompt_template = """Answer the question based only on the following context:

{context}

Question: {question}

Answer in clear, concise English. If unsure, say "I don't know"."""


In [74]:


vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="pdf-qa"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [75]:
prompt = ChatPromptTemplate.from_template(prompt_template)
rag_chain = (
    {"context": retriever | (lambda docs: truncate_context("\n\n".join(d.page_content for d in docs))),
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [76]:
def ask_with_details(query: str):
    try:
        context_docs = retriever.invoke(query)


        print(f"\n{'='*50}\nQuery: {query}")
        print(f"Retrieved {len(context_docs)} documents:")

        for i, doc in enumerate(context_docs):
            similarity = cosine_similarity(
                embeddings.embed_query(query),
                embeddings.embed_query(doc.page_content)
            )

        answer = rag_chain.invoke(query)
        print(f"\nAnswer: {answer}\n{'='*50}\n")

    except Exception as e:
        print(f"Error processing query: {str(e)}")


In [81]:
ask_with_details("What is Artificial Intelligence?")



Query: What is Artificial Intelligence?
Retrieved 2 documents:

Answer: Science and engineering of making intelligent machines

